## Applied Data Science Capstone Week 3 - Segmenting and Clustering Neighborhoods in Toronto

## 4. Submit a link to your Notebook on your Github repository. (10 marks)
### Parse the webpage using Beautiful Soup

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scrape the page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Request page
response = requests.get(url)

# Read page content
soup = BeautifulSoup(response.content)

# Get the data from the rows (and skip header)
data_raw = soup.select(".wikitable tr:not(:first-child)")

### Clean the data and setup the dataframe

In [57]:
all_data = {"PostalCode": [], "Borough": [], "Neighborhood": []}
i = 0

# Loop all rows
for row in data_raw:
    tds = row.select("td")
    
    # Get row values
    postalcode = tds[0].get_text().strip()
    borough = tds[1].get_text().strip()
    neighborhood = tds[2].get_text().strip()
    
    # Skip where Borough = "Not assigned"
    if borough == "Not assigned":
        continue
    
    # One row per Postal Code
    if postalcode not in all_data["PostalCode"]:
        all_data["PostalCode"].append(postalcode)
        all_data["Neighborhood"].append(neighborhood)
        
        # One borough per Postal Code
        if borough not in all_data:
            all_data["Borough"].append(borough)
    else:
        # Concatenate the "Neighborhood" values
        index_postalcode = all_data["PostalCode"].index(postalcode)
        all_data["Neighborhood"][index_postalcode] +=  ", " + neighborhood

# Dictionary with the data to pass to a dataframe
data = {
    "PostalCode": all_data["PostalCode"],
    "Borough": all_data["Borough"],
    "Neighborhood": all_data["Neighborhood"]
}

# Setup Dataframe from the dictionary
df = pd.DataFrame(data)

# Replace "Neighborhood" columns having "Not assigned" with the same value as the Borough in the same row
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

### Dataframe shape

In [58]:
df.shape

(103, 3)

### Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

### Read the data from the Geospatial Data CSV

In [70]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_3eb71cfbc3f443e798d46bcbbbb96002 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='gLHKKtP9PBdbvpA3go_ZBUr6lw_F6n625M6wUkwE8Rbm',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3eb71cfbc3f443e798d46bcbbbb96002.get_object(Bucket='datasciencecapstoneweek2-donotdelete-pr-g8h19o8qhwqhf4',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_geospatial = pd.read_csv(body)

### Merge the Geospatial dataframe with the neighborhoods dataframe

In [71]:

df_final = df_geospatial.merge(df, left_on= "Postal Code", right_on="PostalCode")
df_final.drop(columns=["Postal Code"], inplace=True)

# Order columns
new_order = [2,3,4,0,1]
df_final = df_final[df_final.columns[new_order]]

df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
